In [1]:
# Python标准库inspect
"""nspect模块用于收集python对象的信息，可以获取类或函数的参数的信息，源码，解析堆栈，对对象进行类型检查等等，有几个好用的方法：

getargspec(func)

返回一个命名元组ArgSpect(args, varargs, keywords, defaults)，args是函数位置参数名列表，varargs是*参数名，keywords是**参数名，defaults是默认参数值的元组。

在用__init__参数自动初始化实例属性的实践中，是用字节码对象的co_varnames属性来获取函数的位置参数名的："""
def attr_from_locals(locals_dict):
    self = locals_dict.pop('self')
    code = self.__init__.__func__.__code__
    args = code.co_varnames[1:code.co_argcount]
    for k in args:
        setattr(self, k, locals_dict[k])
        
class Foo(object):
    def __init__(self, name, color, num=1):
        x = 1
        attr_from_locals(locals())
"""而当__init__方法使用**特殊参数接收任意数量的关键字参数时，上述代码是不适用的。可行的办法是使用字节码的co_flags属性来判断**参数是否存在。

函数使用*args语法来接受任意数量的位置参数时，co_flags置位0x04，使用**kwargs语法时，置位0x08，函数为一个生成器时，置位0x2000，其它位保留："""
def foo(x, *args, **kwargs):
    pass

In [2]:
foo.__code__.co_varnames

('x', 'args', 'kwargs')

In [3]:
foo.__code__.co_flags & 0x04

4

In [4]:
foo.__code__.co_flags & 0x08

8

In [6]:
"""inspect模块的getargspec()方法正是用此判断来获取函数的特殊参数的。现在可以方便的获取__init__的**参数了："""
import inspect
 
 
def attr_from_locals(locals_dict):
    self = locals_dict.pop('self')
    args = inspect.getargspec(self.__init__.__func__).args[1:]
    for k in args:
        setattr(self, k, locals_dict[k])
    keywords = inspect.getargspec(self.__init__.__func__).keywords
    if keywords:
        keywords_dict = locals_dict[keywords]
        for k in keywords_dict:
            setattr(self, k, keywords_dict[k])
 
        
class Foo(object):
    def __init__(self, name, **kwargv):
        attr_from_locals(locals())
 
 
f = Foo('bar', color='yellow', num=1)
print(f.__dict__)

{'name': 'bar', 'color': 'yellow', 'num': 1}


D:\Program Files\Anaconda3.5.1.0\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  import sys
D:\Program Files\Anaconda3.5.1.0\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  # Remove the CWD from sys.path while we load stuff.
